In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/colab/colab_ds.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/colab/df_train.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/colab/df_test.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/colab/df_val.pkl" /content/
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
!unzip colab_ds.zip

Streaming output truncated to the last 5000 lines.
   creating: colab_ds/Rhyacophila_nub103/
 extracting: colab_ds/Rhyacophila_nub103/0-Rhyacophila_nub103.1.2016-06-54-43-381622000000-utc.png  
   creating: colab_ds/Rhyacophila_nub104/
 extracting: colab_ds/Rhyacophila_nub104/0-Rhyacophila_nub104.1.2016-06-55-19-419915000000-utc.png  
 extracting: colab_ds/Rhyacophila_nub104/1-Rhyacophila_nub104.1.2016-06-55-19-932956000000-utc.png  
 extracting: colab_ds/Rhyacophila_nub104/1-Rhyacophila_nub104.1.2016-06-55-20-246182000000-utc.png  
   creating: colab_ds/Rhyacophila_nub105/
 extracting: colab_ds/Rhyacophila_nub105/1-Rhyacophila_nub105.1.2016-06-55-37-567349000000-utc.png  
 extracting: colab_ds/Rhyacophila_nub105/1-Rhyacophila_nub105.1.2016-06-55-37-671155000000-utc.png  
 extracting: colab_ds/Rhyacophila_nub105/1-Rhyacophila_nub105.1.2016-06-55-38-064093000000-utc.png  
   creating: colab_ds/Rhyacophila_nub106/
 extracting: colab_ds/Rhyacophila_nub106/0-Rhyacophila_nub106.1.2016-06-56

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_tf_dataset, prepare_for_training

df_train = np.load('df_train.pkl',allow_pickle=True)
df_test = np.load('df_test.pkl',allow_pickle=True)
df_val = np.load('df_val.pkl',allow_pickle=True)

def fix_path(fpath):
  path, file_ = ntpath.split(fpath)
  _, folder = ntpath.split(path)
  
  return os.path.join('colab_ds',folder,file_)

df_train['path'] = df_train['path'].apply(fix_path)
df_test['path'] = df_test['path'].apply(fix_path)
df_val['path'] = df_val['path'].apply(fix_path)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_val = shuffle(df_val)

df_train.head()

,path,label
283282,colab_ds/Simuliidae205/1-Simuliidae205.1.2016-...,37
227441,colab_ds/Neureclipsis_bim302/1-Neureclipsis_bi...,27
218180,colab_ds/Nemoura_fle71/0-Nemoura_fle71.1.2016-...,26
255237,colab_ds/Plectronemia_con1/0-Plectronemia_con1...,30
160702,colab_ds/Leptophlebia164/1-Leptophlebia164.1.2...,19


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 32

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)
    print(label)


(32, 224, 224, 3)
(32, 39)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 39), dtype=float32)
(32, 224, 224, 3)
(32, 39)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 39), dtype=float32)
(32, 224, 224, 3)
(32, 39)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 39), dtype=float32)
(32, 224, 224, 3)
(32, 39)
tf.Tensor(
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 39), dtype=float32)
(32, 224, 224, 3)
(32, 39)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

import tensorflow_addons as tfa


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()


focal = tfa.losses.SigmoidFocalCrossEntropy()

model.compile(optimizer = 'adam', loss = focal,
                  metrics=['accuracy'])


87916544/87910968 [==============================] - 3s 0us/step


In [ ]:
tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 30,
          validation_steps = val_steps)

Train for 502 steps, validate for 71 steps
Epoch 1/30
502/502 [==============================] - 98s 196ms/step - loss: 0.4250 - accuracy: 0.4239 - val_loss: 0.4946 - val_accuracy: 0.4313
Epoch 2/30
502/502 [==============================] - 81s 162ms/step - loss: 0.2278 - accuracy: 0.6522 - val_loss: 0.3731 - val_accuracy: 0.5335
Epoch 3/30
502/502 [==============================] - 81s 162ms/step - loss: 0.1700 - accuracy: 0.7290 - val_loss: 0.2452 - val_accuracy: 0.6224
Epoch 4/30
502/502 [==============================] - 81s 162ms/step - loss: 0.1317 - accuracy: 0.7875 - val_loss: 0.2770 - val_accuracy: 0.6382
Epoch 5/30
502/502 [==============================] - 81s 162ms/step - loss: 0.1142 - accuracy: 0.8190 - val_loss: 0.2892 - val_accuracy: 0.6193
Epoch 6/30
502/502 [==============================] - 81s 162ms/step - loss: 0.0949 - accuracy: 0.8469 - val_loss: 0.1935 - val_accuracy: 0.7289
Epoch 7/30
502/502 [==============================] - 81s 162ms/step - loss: 0.0810 - a